In [1]:
import pandas as pd

%load_ext autoreload
%autoreload 2

In [2]:
%cd ..

/home/vmadmin/project


# Create model-generated data:

In [ ]:
from company_generator import CompanyGenerator

In [ ]:
wg = CompanyGenerator('models/gpt2_forward_model', blacklist_path='data/company_blacklist.pickle', industries_path='data/industries.pickle')

In [ ]:
import numpy as np

gen_comps, gen_stats = wg.generate_companies(num=50000, max_iterations=np.inf, generation_args=dict(top_k=300, num_return_sequences=48, max_length=512, do_sample=True), save_path='data/generated_companies.csv')

# Create tfidf&random-generated data:

In [3]:
from weak_generator import WeakFakeGenerator

In [4]:
wg = WeakFakeGenerator('data/all_data.csv', save_cache='models/tfidf_model/tfidf_model_corpus_dict_industries.pickle')

Loading tfidf model from path... done.


In [11]:
# tfidf & random are generated using a dataframe of existing companies
# we use the model-generated companies for that:
strong_fake = pd.read_csv('data/strong_generated_companies.csv')
strong_fake

,company_name,industry,text
0,all people weds,hospital & health care,Home - All People Weds About Leadership Initia...
1,infinity travel,hospitality,"We Travel Here, Entertain Here, Entertain Here..."
2,clark marine maintenance & refrigeration,consumer services,Clark Marine Maintenance & Refrigeration | Ser...
3,mackenville eye supplies,automotive,Mackenville Eye Supplies | Optical | Cosmetic ...
4,tecpa finance ltd,construction,Home Page | Tecpa Finance Ltd Home What we off...
...,...,...,...
49995,go financial strategies,financial services,Go Financial Strategies HOME - GO Financial St...
49996,finchpro ltd,entertainment,Home - FinchPro Skip to content Search for... ...
49997,abibc,accounting,Accountant Broker | Brisbane | Abibc Australia...
49998,smartly connect,printing,Smartly Connect home about services contact ho...


In [10]:
# create and save the tfidf & random
wg.generate(strong_fake, tfidf=True, save_path='data/weak_tfidf.csv')
wg.generate(strong_fake, tfidf=False, save_path='data/weak_random.csv')

# Create and save training and testing data from each of the generated datasets

In [12]:
from utils import clean_text

real_all = pd.read_csv('data/all_data.csv', index_col=0, nrows=50000, lineterminator='\n')
real_all['text'] = real_all.text.apply(clean_text)
real_all['company_name'] = real_all.company_name.apply(lambda s: s.replace('-', ' '))
weak_tfidf = pd.read_csv('data/weak_tfidf.csv', index_col=0)
weak_random = pd.read_csv('data/weak_random.csv', index_col=0)

real_all['label'] = False
for fake in [strong_fake, weak_tfidf, weak_random]:
    fake['label'] = True

In [13]:
real_all_train, real_all_test = real_all.iloc[:40000], real_all.iloc[40000:]
weak_tfidf_train, weak_tfidf_test = weak_tfidf.iloc[:40000], weak_tfidf.iloc[40000:]
weak_random_train, weak_random_test = weak_random.iloc[:40000], weak_random.iloc[40000:]
strong_fake_train, strong_fake_test = strong_fake.iloc[:40000], strong_fake.iloc[40000:]

In [9]:
pd.concat([real_all_train, weak_tfidf_train], axis=0, ignore_index=True).to_csv('data/classifier_data/tfidf_real_train.csv', index=False)
pd.concat([real_all_test, weak_tfidf_test], axis=0, ignore_index=True).to_csv('data/classifier_data/tfidf_real_test.csv', index=False)
pd.concat([real_all_train, weak_random_train], axis=0, ignore_index=True).to_csv('data/classifier_data/random_real_train.csv', index=False)
pd.concat([real_all_test, weak_random_test], axis=0, ignore_index=True).to_csv('data/classifier_data/random_real_test.csv', index=False)
pd.concat([real_all_train, strong_fake_train], axis=0, ignore_index=True).to_csv('data/classifier_data/generated_real_train.csv', index=False)
pd.concat([real_all_test, strong_fake_test], axis=0, ignore_index=True).to_csv('data/classifier_data/generated_real_test.csv', index=False)